In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/allstate-claims-severity/sample_submission.csv
/kaggle/input/allstate-claims-severity/train.csv.zip
/kaggle/input/allstate-claims-severity/sample_submission.csv.zip
/kaggle/input/allstate-claims-severity/train.csv
/kaggle/input/allstate-claims-severity/test.csv
/kaggle/input/allstate-claims-severity/test.csv.zip


In [2]:
import pandas as pd

df_train = pd.read_csv('/kaggle/input/allstate-claims-severity/train.csv')
df_test = pd.read_csv('/kaggle/input/allstate-claims-severity/test.csv')

print (df_train.shape)
print (df_test.shape)

(188318, 132)
(125546, 131)


In [3]:
from IPython.display import display
print (df_train.columns)

display(df_train.head())

Index(['id', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8',
       'cat9',
       ...
       'cont6', 'cont7', 'cont8', 'cont9', 'cont10', 'cont11', 'cont12',
       'cont13', 'cont14', 'loss'],
      dtype='object', length=132)


,id,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,loss
0,1,A,B,A,B,A,A,A,A,B,...,0.718367,0.335060,0.30260,0.67135,0.83510,0.569745,0.594646,0.822493,0.714843,2213.18
1,2,A,B,A,A,A,A,A,A,B,...,0.438917,0.436585,0.60087,0.35127,0.43919,0.338312,0.366307,0.611431,0.304496,1283.60
2,5,A,B,A,A,B,A,A,A,B,...,0.289648,0.315545,0.27320,0.26076,0.32446,0.381398,0.373424,0.195709,0.774425,3005.09
3,10,B,B,A,B,A,A,A,A,B,...,0.440945,0.391128,0.31796,0.32128,0.44467,0.327915,0.321570,0.605077,0.602642,939.85
4,11,A,B,A,B,A,A,A,A,B,...,0.178193,0.247408,0.24564,0.22089,0.21230,0.204687,0.202213,0.246011,0.432606,2763.85


# Checking the categorical features

In [4]:
categorical_columns = df_train.select_dtypes(include=['object']).columns
print ("Categorical Column: ", categorical_columns)

Categorical Column:  Index(['cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8', 'cat9',
       'cat10',
       ...
       'cat107', 'cat108', 'cat109', 'cat110', 'cat111', 'cat112', 'cat113',
       'cat114', 'cat115', 'cat116'],
      dtype='object', length=116)


# Finding duplicates and NaN values

In [5]:
df_train.duplicated().sum() #no duplicates
df_train.isnull().sum() #no NaN values

df_test.duplicated().sum()
df_test.isnull().sum()

id        0
cat1      0
cat2      0
cat3      0
cat4      0
         ..
cont10    0
cont11    0
cont12    0
cont13    0
cont14    0
Length: 131, dtype: int64

In [6]:
x_train = df_train.iloc[:,:-1]
y_train = df_train.iloc[:,-1]

In [7]:
display (x_train.head())
display(y_train.head())

print (x_train.shape)
print (y_train.shape)

,id,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14
0,1,A,B,A,B,A,A,A,A,B,...,0.310061,0.718367,0.335060,0.30260,0.67135,0.83510,0.569745,0.594646,0.822493,0.714843
1,2,A,B,A,A,A,A,A,A,B,...,0.885834,0.438917,0.436585,0.60087,0.35127,0.43919,0.338312,0.366307,0.611431,0.304496
2,5,A,B,A,A,B,A,A,A,B,...,0.397069,0.289648,0.315545,0.27320,0.26076,0.32446,0.381398,0.373424,0.195709,0.774425
3,10,B,B,A,B,A,A,A,A,B,...,0.422268,0.440945,0.391128,0.31796,0.32128,0.44467,0.327915,0.321570,0.605077,0.602642
4,11,A,B,A,B,A,A,A,A,B,...,0.704268,0.178193,0.247408,0.24564,0.22089,0.21230,0.204687,0.202213,0.246011,0.432606


0    2213.18
1    1283.60
2    3005.09
3     939.85
4    2763.85
Name: loss, dtype: float64

(188318, 131)
(188318,)


# Feature Engineering
### Categorical features

In [8]:
# column 1 to 117 all are categorical columns
# lets convert the categorical columns to one hot encoding

print (x_train.iloc[:,1:117].astype('category'))

       cat1 cat2 cat3 cat4 cat5 cat6 cat7 cat8 cat9 cat10  ... cat107 cat108  \
0         A    B    A    B    A    A    A    A    B     A  ...      J      G   
1         A    B    A    A    A    A    A    A    B     B  ...      K      K   
2         A    B    A    A    B    A    A    A    B     B  ...      F      A   
3         B    B    A    B    A    A    A    A    B     A  ...      K      K   
4         A    B    A    B    A    A    A    A    B     B  ...      G      B   
...     ...  ...  ...  ...  ...  ...  ...  ...  ...   ...  ...    ...    ...   
188313    A    B    A    A    A    A    A    A    B     A  ...      G      B   
188314    A    A    A    A    A    B    A    A    A     A  ...      F      B   
188315    A    B    A    A    A    A    A    B    B     A  ...      F      B   
188316    A    B    A    A    A    A    A    A    B     B  ...      J      K   
188317    B    A    A    B    A    A    A    A    A     A  ...      G      G   

       cat109 cat110 cat111 cat112 cat1

In [9]:
categorical_columns = x_train.select_dtypes(include = ['object','category']).columns
x_train = pd.get_dummies(x_train, columns = categorical_columns, drop_first = True)

In [10]:
display (x_train.head())

,id,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,...,cat116_P,cat116_Q,cat116_R,cat116_S,cat116_T,cat116_U,cat116_V,cat116_W,cat116_X,cat116_Y
0,1,0.726300,0.245921,0.187583,0.789639,0.310061,0.718367,0.335060,0.30260,0.67135,...,False,False,False,False,False,False,False,False,False,False
1,2,0.330514,0.737068,0.592681,0.614134,0.885834,0.438917,0.436585,0.60087,0.35127,...,False,False,False,False,False,False,False,False,False,False
2,5,0.261841,0.358319,0.484196,0.236924,0.397069,0.289648,0.315545,0.27320,0.26076,...,False,False,False,False,False,False,False,False,False,False
3,10,0.321594,0.555782,0.527991,0.373816,0.422268,0.440945,0.391128,0.31796,0.32128,...,False,False,False,False,False,False,False,False,False,False
4,11,0.273204,0.159990,0.527991,0.473202,0.704268,0.178193,0.247408,0.24564,0.22089,...,False,False,False,False,False,False,False,False,False,False


In [11]:
print (x_train.dtypes.unique())
float_columns = x_train.select_dtypes(include = ['float64'])
print (float_columns.max())
print (float_columns.min())
# we can conclude the numerical values are already min max scaled 
# so don't need to pre-process this data

[dtype('int64') dtype('float64') dtype('bool')]
cont1     0.984975
cont2     0.862654
cont3     0.944251
cont4     0.954297
cont5     0.983674
cont6     0.997162
cont7     1.000000
cont8     0.980200
cont9     0.995400
cont10    0.994980
cont11    0.998742
cont12    0.998484
cont13    0.988494
cont14    0.844848
dtype: float64
cont1     0.000016
cont2     0.001149
cont3     0.002634
cont4     0.176921
cont5     0.281143
cont6     0.012683
cont7     0.069503
cont8     0.236880
cont9     0.000080
cont10    0.000000
cont11    0.035321
cont12    0.036232
cont13    0.000228
cont14    0.179722
dtype: float64


# Train Test Split

In [12]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split (x_train, y_train, test_size = 0.3, 
                                                    random_state = 42)

print (x_train.shape)
print (x_test.shape)
print (y_train.shape)
print (y_test.shape)

(131822, 1038)
(56496, 1038)
(131822,)
(56496,)
